In [1]:
# Imports
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt
import csv
import random

In [2]:
reclustered_monthly_data = pd.read_csv("output_hpc/reclustered_monthly_data_final.csv", sep = "\t", encoding = "utf-8", quoting = csv.QUOTE_NONE )

In [ ]:
data = reclustered_monthly_data.head(100)

In [74]:
n_rows = data.shape[0]

In [75]:
data["A"] = np.random.rand(n_rows)
data["F"] = np.random.rand(n_rows)
data["N"] = np.random.rand(n_rows)

/tmp/ipykernel_9866/686247784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["A"] = np.random.rand(n_rows)
/tmp/ipykernel_9866/686247784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["F"] = np.random.rand(n_rows)
/tmp/ipykernel_9866/686247784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [76]:
data_avg = data.groupby("user")[['A', 'F', 'N']].mean().reset_index()

In [77]:
max_val = data_avg.iloc[1][["A", "F", "N"]].max()
max_col = data_avg.iloc[1][["A", "F"]].idxmax()
print(max_val *-1)
print(max_val, max_col)

-0.974251290632651
0.974251290632651 F


In [5]:
def get_stance(row): 
    if row["N"] == 1.0: 
        return 0
    elif row["A"] > row["F"]:
        return row["A"]
    elif row["F"] > row["A"]:
        return row["F"]*-1
    else: 
        return random.choice([row["A"], row["F"]])

In [79]:
data_avg["stance"] = data_avg.apply(lambda x: get_stance(x), axis = 1)

#### Getting the stance for each user for each topic

In [10]:
def make_dataset(data):
    data = reclustered_monthly_data.head(100)
    n_rows = data.shape[0]
    data["A"] = np.random.rand(n_rows)
    data["F"] = np.random.rand(n_rows)
    data["N"] = np.random.rand(n_rows)
    data_avg = data.groupby("user")[['A', 'F', 'N']].mean().reset_index()
    data_avg["stance"] = data_avg.apply(lambda x: get_stance(x), axis = 1)
    return data_avg

In [13]:
data_1 = make_dataset(reclustered_monthly_data)
data_2 = make_dataset(reclustered_monthly_data)
data_3 = make_dataset(reclustered_monthly_data)

/tmp/ipykernel_869/2471956829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["A"] = np.random.rand(n_rows)
/tmp/ipykernel_869/2471956829.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["F"] = np.random.rand(n_rows)
/tmp/ipykernel_869/2471956829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [23]:
data1 = data_1.rename(columns={"stance": "stance_1"})
data2 = data_2.rename(columns={"stance": "stance_2"})
data3 = data_3.rename(columns={"stance": "stance_3"})

In [27]:
merged_df = pd.merge(pd.merge(data1, data2, on='user'), data3, on='user')

In [31]:
merged_df = merged_df[["user", "stance_1", "stance_2", "stance_3"]]
merged_df

,user,stance_1,stance_2,stance_3
0,AlbionPrince,0.931657,-0.954651,-0.880233
1,BenFoldsFourLoko,0.408146,-0.705791,-0.519300
2,Betrashndie,-0.935226,-0.722485,-0.200872
3,BloodWiz,-0.473356,0.480935,0.778890
4,Call_Me_Hurr1cane,-0.681776,0.433288,-0.877143
...,...,...,...,...
91,thebigslapper,0.988245,0.494813,-0.624964
92,throwrowrowawayyy,0.863324,0.846066,-0.979942
93,trogon,0.622945,0.248133,0.751745
94,weeteacups,-0.778994,-0.314035,-0.754830


#### Dimensionality reduction

##### Computing polarization 